In [ ]:
import sys
sys.path.append('../..')

from core.fine_model import FineModel
from keras.models import Sequential

import os
import json
import math

import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import sklearn
import numpy as np
import pandas as pd
from tqdm import tqdm

import cr_interface as cri
import keras_utils as ku
import keras_bottle as kb
import keras_history as kh
import keras_weights as kw
import lib

In [ ]:
batch_size = 32

In [ ]:
models = FineModel.get_dict()

In [ ]:
models.keys()

In [ ]:
fm = models['resnet50']()

In [ ]:
aug_gen = fm.get_image_data_generator(augment=True)
pure_gen = fm.get_image_data_generator(augment=False)

In [ ]:
train_collection = cri.CrCollection.load().filter_by(dataset_index=0).tri_label().labeled()
test_collection = cri.CrCollection.load().filter_by(dataset_index=1).tri_label().labeled()

train_collection = train_collection.sample(frac=1)
test_collection = test_collection.sample(frac=0.1)

train, validation = train_collection.split_by('pid', [0.8, 0.2])
test = test_collection

In [ ]:
name = fm.get_name()
train_dir = '{}_train'.format(name)
validation_dir = '{}_validation'.format(name)
test_dir = '{}_test'.format(name)

In [ ]:
train.export_by_label(train_dir)
validation.export_by_label(validation_dir)
test.export_by_label(test_dir, balancing=1)

In [ ]:
train_gen = aug_gen.flow_from_directory(
    train_dir,
    target_size=fm.get_output_shape(),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
validation_gen = aug_gen.flow_from_directory(
    validation_dir,
    target_size=fm.get_output_shape(),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
test_gen = pure_gen.flow_from_directory(
    test_dir,
    target_size=fm.get_output_shape(),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
fm.get_depths()

In [ ]:
epochs = 5

In [ ]:
fm.set_depth(0)
fm.compile_model(lr=1e-4)
fm.get_model().fit_generator(
    train_gen,
    validation_data=validation_gen,
    steps_per_epoch=math.ceil(train.df.shape[0] / batch_size),
    validation_steps=1,
    shuffle=True,
    epochs=epochs)

In [ ]:
fm.get_model().evaluate_generator(
    test_gen,
    steps=1
)

In [ ]:
fm.set_depth(1)
fm.compile_model(lr=1e-4)
fm.get_model().fit_generator(
    train_gen,
    validation_data=validation_gen,
    steps_per_epoch=math.ceil(train.df.shape[0] / batch_size),
    validation_steps=1,
    shuffle=True,
    epochs=epochs)

In [ ]:
fm.get_model().evaluate_generator(
    test_gen,
    steps=1
)